In this notebook we want to test automatising the computation of matrix elements as functions of position, and F^H (so to be able to use Krylov iterative solvers), permanent storage in an according file containing all symbolic results (this will only be generated once per Green's function), and the subsequent usage of this file by the linear solver.

On the note of a general Green's function, have to write the matrix elements more general. In previous computation I have used a lot of the symmetries of the free space Green's function.

In [1]:
from sympy import *

In [64]:
xij, yij, zij, b, eta = symbols("xij yij zij b eta", real=True) #probably need FH and VH symbols too

# indices for summation:
## Caution, cannot use lambda, instead use Lambda
## do not use eta as index, it's the viscosity
alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma = symbols(
    'alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma', integer=True)


## for FH and VH checkout the 'Indexed' package to define them as FH[1], FH[2] etc.

rij = Array([xij, yij, zij])

def norm(rij):
    return sqrt(summation(rij[alpha]**2,(alpha,0,2)))

KroneckerDelta = eye(3)

def Laplacian(G, rij):
    return diff(G, rij[0], rij[0]) + diff(G, rij[1], rij[1]) + diff(G, rij[2], rij[2])

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

In [3]:
norm(rij)

sqrt(xij**2 + yij**2 + zij**2)

In [4]:
def G(rij, alpha, beta):
    return 1/(8*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij) + rij[alpha]*rij[beta]/norm(rij)**3)

def delG(rij, alpha, beta, gamma):
    return 1/(8*pi*eta)*((-KroneckerDelta[alpha,beta]*rij[gamma] + KroneckerDelta[alpha,gamma]*rij[beta] + KroneckerDelta[beta,gamma]*rij[alpha])/norm(rij)**3
                         - 3*rij[alpha]*rij[beta]*rij[gamma]/norm(rij)**5)

def lapG(rij, alpha, beta):
    return 1/(4*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij)**3 - 3*rij[alpha]*rij[beta]/norm(rij)**5)

In [10]:
def delG_symb(rij, alpha, beta, gamma):
    return diff(G(rij, alpha, beta), rij[gamma])

def lapG_symb(rij, alpha, beta):
    return Laplacian(G(rij, alpha, beta), rij)

In [11]:
simplify(delG(rij,0,0,1)) - simplify(delG_symb(rij, 0,0,1))

0

In [14]:
simplify(lapG(rij, 0,1)) - simplify(lapG_symb(rij, 0,1))

0

In [28]:
##auxiliary functions
def dellapG(rij, alpha, beta, kappa1):
    return diff(lapG(rij, alpha, beta), rij[kappa1])

def deldelG(rij, zeta, beta, kappa1, nu):
    return diff(delG(rij, zeta, beta, kappa1), rij[nu])

def deldellapG(rij, alpha, beta, gamma1, kappa1):
    return diff(dellapG(rij,alpha,beta, gamma1), rij(kappa1))

def deldeldelG(rij, alpha, beta, gamma1, gamma2, gamma3):
    return diff(deldelG(rij, alpha, beta, gamma1, gamma2), rij(gamma3))

def deldeldeldelG(rij, alpha, beta, gamma1, gamma2, gamma3, gamma4):
    return diff(deldeldelG(rij, alpha, beta, gamma1, gamma2, gamma3), rij(gamma4))

In [66]:
# GLL
def G1s1s(rij, alpha, beta):
    return G(rij, alpha, beta) + b**2/3*lapG(rij, alpha, beta)

def G1s2a(rij, alpha, beta):
    return -0.5*b*summation(summation(eps(beta, nu, zeta)*delG(rij, alpha, zeta, nu), (nu,0,2)), (zeta,0,2))

def G2a1s(rij, alpha, beta):
    return b*summation(summation(eps(alpha,nu,zeta)*delG(rij,zeta,beta,nu), (nu,0,2)), (zeta,0,2))

def G2a2a(rij, alpha, beta):
    return -0.5*b*b*summation(summation(summation(summation(eps(alpha,mu,kappa)*eps(beta,nu,zeta)*deldelG(rij,kappa,zeta,nu,mu),(mu,0,2)), (kappa,0,2)), (nu,0,2)), (zeta,0,2))


## GL2s and G2sL
def G1s2s(rij, alpha, kappa1, beta):
    return -0.5*b*((delG(rij, alpha, beta, kappa1) + delG(rij, alpha, kappa1, beta)) + 4*b**2/15*(dellapG(rij, alpha, beta, kappa1) + dellapG(rij, alpha, kappa1, beta)))

def G2a2s(rij, alpha, kappa1, beta):
    return -0.5*b*b*summation(summation(eps(alpha,nu,zeta)*(deldelG(rij, zeta, beta, kappa1, nu) + deldelG(rij, zeta, kappa1, beta, nu)), (nu,0,2)), (zeta,0,2))

def G2s1s(rij, alpha, gamma1, beta):
    return 0.5*b*((delG(rij, alpha, beta, gamma1) + delG(rij, gamma1, beta, alpha)) + 4*b**2/15*(dellapG(rij,alpha,beta,gamma1) + dellapG(rij, gamma1, beta, alpha)))

def G2s2a(rij, alpha, gamma1, mu):
    return 0.25*b*b*summation(summation(eps(beta, kappa1, mu)*(deldelG(rij, gamma1, beta, alpha, kappa1) + deldelG(rij, alpha, beta, gamma1, kappa1)), (kappa1, 0, 2)), (beta,0,2))


## G2s2s
def G2s2s(rij, alpha, gamma1, kappa1, beta):
    return -0.25*b*b*((deldelG(rij, alpha, beta, gamma1, kappa1) + deldelG(rij, gamma1, beta, alpha, kappa1)) + 
                      (deldelG(rij,alpha,kappa1, gamma1, beta) + deldelG(rij, gamma1, kappa1, alpha, beta)) +
                      b**2/5*(deldellapG(rij,alpha,beta,gamma1,kappa1) + deldellapG(rij,gamma1,beta,alpha,kappa1)) + 
                      b**2/5*(deldellapG(rij, alpha, kappa1, gamma1, beta) + deldellapG(rij,gamma1,kappa1,alpha,beta)))


## higher matrix elements
def G1s3t(rij, alpha, beta):
    return -1/5*b**2*lapG(rij,alpha,beta)


# symmetry for matrix elements of form l sigma, 3s
def g13sSym(rij,alpha,kappa2,kappa1,beta):
    return 1/3*(deldelG(rij,alpha,beta,kappa1,kappa2) + deldelG(rij,alpha,kappa2,kappa1,beta) + deldelG(rij,alpha,kappa1,beta,kappa2)) - 1/15*(
        KroneckerDelta[kappa1,kappa2]*lapG(rij,alpha,beta) + KroneckerDelta[kappa1,beta]*lapG(rij,alpha,kappa2) + KroneckerDelta[beta,kappa2]*lapG(rij,alpha,kappa1))

def G1s3s(rij,alpha,kappa2, kappa1, beta):
    return b**2*(g13sSym(rij,alpha,kappa2,kappa1,beta) + 5*b**2/21*Laplacian(g13sSym(rij,alpha,kappa2,kappa1,beta),rij))


# symmetry of matrix elements of form 3s, l sigma
def g3s1Sym(rij,alpha,gamma1,gamma2,beta):
    return 1/3*(deldelG(rij,alpha,beta,gamma1,gamma2) + deldelG(rij,gamma1,beta,gamma2,alpha) + deldelG(rij,gamma2,beta,alpha,gamma1)) - 1/15*(
        KroneckerDelta[gamma1, gamma2]*lapG(rij,alpha,beta) + KroneckerDelta[gamma2,alpha]*lapG(rij,gamma1,beta) + KroneckerDelta[alpha,gamma1]*lapG(rij,gamma2,beta))

def G3s1s(rij,alpha,gamma1,gamma2,beta):
    return b**2*(g3s1Sym(rij,alpha,gamma1,gamma2,beta) + 5*b**2/21*Laplacian(g3s1Sym(rij,alpha,gamma1,gamma2,beta),rij))

def G3a1s(rij,Lambda, kappa2, beta):
    return 0.5*b**2*summation(summation(eps(Lambda, gamma1, alpha)*deldelG(rij,alpha,beta,gamma1,kappa2) +
                                        eps(kappa2, gamma1, alpha)*deldelG(rij,alpha,beta,gamma1,Lambda), (gamma,0,2)), (alpha,0,2))

def G1s3a(rij, alpha, nu2, sigma):
    return -2/3*b**2*summation(summation(eps(beta,nu1,sigma)*(deldelG(rij,alpha,beta,nu1,nu2) - 1/3*KroneckerDelta[nu1,nu2]*lapG(rij, alpha,beta)), (beta,0,2)), (nu1,0,2))

def G3s2a(rij, alpha,gamma1,gamma2,mu):
    return 0.5*b**3*summation(summation(eps(beta,kappa1,mu)))## diff and then sum over it will not work need function that can be summed over with epsilon etc like curl
#0.5*b^3*Sum[Sum[eps[beta, kappa1, mu]*D[g3s1Sym[rij, alpha, gamma1, gamma2, beta], rij[kappa1]], {kappa1, 3}], {beta, 3}]

In [68]:
simplify(Laplacian(G(rij,0,1),rij))

-3*xij*yij/(4*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))

In [79]:
summation(summation(diff(eps(beta,1,alpha)*G(rij,0,beta), rij[alpha]), (alpha,0,2)), (beta,0,2)) ## different result for mathematica

0

In [72]:
i

2

In [41]:
## try to assemble one of the matrices

In [60]:
matG1s1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s1s[i,j] = simplify(G1s1s(rij, i, j))

In [61]:
matG1s1s ##works and as far as I can see, matches the Mathematica result

Matrix([
[(-2*b**2*(2*xij**2 - yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(2*xij**2 + yij**2 + zij**2))/(24*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)),                                                         xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)),                                                         xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))],
[                                                       xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (-2*b**2*(-xij**2 + 2*yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(xij**2 + 2*yij**2 + zij**2))/(24*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)),                                                         yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))],
[                                                       xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*

In [ ]:
## when printing to python code, might have to be careful with greek characters that sympy is printing so nicely